In [1]:
import os
import sys

MODULE_PATH = '/content/drive/MyDrive/GitHub/DL_Study/Base'

sys.path.insert(0, MODULE_PATH)
sys.path

['/content/drive/MyDrive/GitHub/DL_Study/Base',
 '',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython']

In [2]:
# import
import numpy
from config import *
from optim import Adam
from models import LstmModelReg

# for time series split
!pip install scikit-learn==0.24.2

In [3]:
# configuration setting
def model_config():
    # parameter for LSTM Model
    epochs = [30]
    batch_size = [32, 64]
    learning_rate = [0.01, 0.001]
    
    # create config data
    configs = []
    for i in epochs:
        for j in batch_size:
            for k in learning_rate:
                config = [i, j, k]
                configs.append(config)
    return configs

# fucntion for fit cnn model using configs
def model_fit(train_X, train_y, config):
    # unpack config
    n_epochs, n_batch, learning_rate = config
    model = LstmModelReg(time_size=24, hidden_size=100, feature_size=9)
    # fit model and return
    model.fit(train_X=train_X, train_y=train_y, epochs=n_epochs, 
              batch_size=n_batch, learning_rate=learning_rate)
    return model

def MAE_metric(x, t):
    return np.mean(np.abs(x-t))

def MSE_metric(x, t):
    return np.mean((x-t)**2)

In [4]:
import pandas as pd
import numpy
import time
from datetime import datetime

np.random.seed(42)
numpy.random.seed(42)

data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00247/'
data_name = 'data_akbilgic.xlsx'
df = pd.read_excel(data_url+data_name, header=1)
df.drop(columns=df.columns[[0]], axis=1, inplace=True)
cols = df.columns.tolist()
cols = cols[2:]+cols[:2]
df = df[cols]
df.head()

,SP,DAX,FTSE,NIKKEI,BOVESPA,EU,EM,ISE,ISE.1
0,-0.004679,0.002193,0.003894,0.000000,0.031190,0.012698,0.028524,0.035754,0.038376
1,0.007787,0.008455,0.012866,0.004162,0.018920,0.011341,0.008773,0.025426,0.031813
2,-0.030469,-0.017833,-0.028735,0.017293,-0.035899,-0.017073,-0.020015,-0.028862,-0.026353
3,0.003391,-0.011726,-0.000466,-0.040061,0.028283,-0.005561,-0.019424,-0.062208,-0.084716
4,-0.021533,-0.019873,-0.012710,-0.004474,-0.009764,-0.010989,-0.007802,0.009860,0.009658


In [5]:
# series data to img function
def series_to_img(dataset, time_step=1):
    num = dataset.shape[1]      # features num
    df = pd.DataFrame(dataset)
    cols, names = list(), list()
    # sequence t-n to t-1
    for i in range(time_step, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(num)]

    for i in range(0, 1):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(num)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(num)]

    agg = pd.concat(cols, axis=1)
    agg.columns = names
    agg.dropna(inplace=True)
    return agg

dataset = df.values
dataset = dataset.astype('float')

n_inputs = 24
n_features = 9
del_idx = n_inputs * n_features + 1
del_cols = [i for i in range(del_idx, del_idx+n_features-1)]
new_df = series_to_img(dataset, n_inputs)
new_df.drop(new_df.columns[del_cols], axis=1, inplace=True)
new_df.head()

,var1(t-24),var2(t-24),var3(t-24),var4(t-24),var5(t-24),var6(t-24),var7(t-24),var8(t-24),var9(t-24),var1(t-23),var2(t-23),var3(t-23),var4(t-23),var5(t-23),var6(t-23),var7(t-23),var8(t-23),var9(t-23),var1(t-22),var2(t-22),var3(t-22),var4(t-22),var5(t-22),var6(t-22),var7(t-22),var8(t-22),var9(t-22),var1(t-21),var2(t-21),var3(t-21),var4(t-21),var5(t-21),var6(t-21),var7(t-21),var8(t-21),var9(t-21),var1(t-20),var2(t-20),var3(t-20),var4(t-20),...,var7(t-5),var8(t-5),var9(t-5),var1(t-4),var2(t-4),var3(t-4),var4(t-4),var5(t-4),var6(t-4),var7(t-4),var8(t-4),var9(t-4),var1(t-3),var2(t-3),var3(t-3),var4(t-3),var5(t-3),var6(t-3),var7(t-3),var8(t-3),var9(t-3),var1(t-2),var2(t-2),var3(t-2),var4(t-2),var5(t-2),var6(t-2),var7(t-2),var8(t-2),var9(t-2),var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var9(t-1),var1(t)
24,-0.004679,0.002193,0.003894,0.000000,0.031190,0.012698,0.028524,0.035754,0.038376,0.007787,0.008455,0.012866,0.004162,0.018920,0.011341,0.008773,0.025426,0.031813,-0.030469,-0.017833,-0.028735,0.017293,-0.035899,-0.017073,-0.020015,-0.028862,-0.026353,0.003391,-0.011726,-0.000466,-0.040061,0.028283,-0.005561,-0.019424,-0.062208,-0.084716,-0.021533,-0.019873,-0.012710,-0.004474,...,0.002243,-0.008777,-0.023458,-0.000533,-0.015637,-0.017454,-0.015134,-0.016289,-0.019739,-0.019091,-0.025919,-0.035607,0.015710,0.024040,0.021039,-0.006175,0.027574,0.017862,0.012719,0.015279,0.022403,-0.007518,0.026577,0.015275,0.026908,0.009565,0.018770,0.015166,0.018578,0.023231,0.016233,0.003932,0.000071,-0.011169,0.024128,-0.004139,0.002073,-0.014133,-0.014571,0.026541
25,0.007787,0.008455,0.012866,0.004162,0.018920,0.011341,0.008773,0.025426,0.031813,-0.030469,-0.017833,-0.028735,0.017293,-0.035899,-0.017073,-0.020015,-0.028862,-0.026353,0.003391,-0.011726,-0.000466,-0.040061,0.028283,-0.005561,-0.019424,-0.062208,-0.084716,-0.021533,-0.019873,-0.012710,-0.004474,-0.009764,-0.010989,-0.007802,0.009860,0.009658,-0.022823,-0.013526,-0.005026,-0.049039,...,-0.019091,-0.025919,-0.035607,0.015710,0.024040,0.021039,-0.006175,0.027574,0.017862,0.012719,0.015279,0.022403,-0.007518,0.026577,0.015275,0.026908,0.009565,0.018770,0.015166,0.018578,0.023231,0.016233,0.003932,0.000071,-0.011169,0.024128,-0.004139,0.002073,-0.014133,-0.014571,0.026541,0.029306,0.014788,0.015846,0.039282,0.019127,0.032338,0.036607,0.042759,0.001484
26,-0.030469,-0.017833,-0.028735,0.017293,-0.035899,-0.017073,-0.020015,-0.028862,-0.026353,0.003391,-0.011726,-0.000466,-0.040061,0.028283,-0.005561,-0.019424,-0.062208,-0.084716,-0.021533,-0.019873,-0.012710,-0.004474,-0.009764,-0.010989,-0.007802,0.009860,0.009658,-0.022823,-0.013526,-0.005026,-0.049039,-0.053849,-0.012451,-0.022630,-0.029191,-0.042361,0.001757,-0.017674,-0.006141,0.000000,...,0.012719,0.015279,0.022403,-0.007518,0.026577,0.015275,0.026908,0.009565,0.018770,0.015166,0.018578,0.023231,0.016233,0.003932,0.000071,-0.011169,0.024128,-0.004139,0.002073,-0.014133,-0.014571,0.026541,0.029306,0.014788,0.015846,0.039282,0.019127,0.032338,0.036607,0.042759,0.001484,0.004766,0.003651,-0.013411,-0.015462,0.005627,0.007895,0.011353,0.021468,-0.050369
27,0.003391,-0.011726,-0.000466,-0.040061,0.028283,-0.005561,-0.019424,-0.062208,-0.084716,-0.021533,-0.019873,-0.012710,-0.004474,-0.009764,-0.010989,-0.007802,0.009860,0.009658,-0.022823,-0.013526,-0.005026,-0.049039,-0.053849,-0.012451,-0.022630,-0.029191,-0.042361,0.001757,-0.017674,-0.006141,0.000000,0.003572,-0.012220,-0.004827,0.015445,-0.000272,-0.034032,-0.047383,-0.050945,0.002912,...,0.015166,0.018578,0.023231,0.016233,0.003932,0.000071,-0.011169,0.024128,-0.004139,0.002073,-0.014133,-0.014571,0.026541,0.029306,0.014788,0.015846,0.039282,0.019127,0.032338,0.036607,0.042759,0.001484,0.004766,0.003651,-0.013411,-0.015462,0.005627,0.007895,0.011353,0.021468,-0.050369,-0.035170,-0.022182,-0.002902,-0.021440,-0.024388,-0.002139,-0.040542,-0.043907,0.007923
28,-0.021533,-0.019873,-0.012710,-0.004474,-0.009764,-0.010989,-0.007802,0.009860,0.009658

In [6]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler

n_splits = 3
test_size = (int)(len(new_df)*0.2)
train_test_split = TimeSeriesSplit(n_splits=n_splits+1, gap=n_inputs, test_size=test_size).split(new_df)
next(train_test_split)

configs = model_config()
history = []
best_error = []
i = 1

print('config : epochs, batch_size, learning_rate')

# neted cross validation
for train_cv_indices, test_cv_indices in train_test_split:
    print(f'fold : {i}/{n_splits}')
    i+=1

    # split x, y data
    train_cv_X, train_cv_y = new_df.iloc[train_cv_indices, :-1].values, new_df.iloc[train_cv_indices,-1].values
    test_cv_X, test_cv_y = new_df.iloc[test_cv_indices, :-1].values, new_df.iloc[test_cv_indices, -1].values

    # length for validation set
    test_length = len(test_cv_X)

    # scaling data
    scaler_x = MinMaxScaler()
    train_cv_X = scaler_x.fit_transform(train_cv_X)
    test_cv_X = scaler_x.transform(test_cv_X)

    train_X, val_X = train_cv_X[:-test_length, :], train_cv_X[-test_length:, :]
    train_y, val_y = train_cv_y[:-test_length], train_cv_y[-test_length:]

    # reshape
    # inner loop
    train_X = train_X.reshape(-1,  n_inputs, n_features)
    val_X = val_X.reshape(-1, n_inputs, n_features)
    train_y = train_y.reshape(-1, 1)
    val_y = val_y.reshape(-1, 1)

    # outer loop
    train_cv_X = train_cv_X.reshape(-1,  n_inputs, n_features)
    test_cv_X = test_cv_X.reshape(-1, n_inputs, n_features)
    train_cv_y = train_cv_y.reshape(-1, 1)
    test_cv_y = test_cv_y.reshape(-1, 1)

    # model fit, inner
    errors = []
    for idx, cfg in enumerate(configs):
        print(f' == train {cfg} model == ', end=' ')
        model = model_fit(train_X, train_y, cfg)
        model.reset_state()
        predicted = model.predict(val_X)
        if GPU:
            predicted = np.asnumpy(predicted)
        error = np.sqrt(MSE_metric(predicted, val_y))   # rmse
        print(f' error(RMSE):{error}')
        if errors:
            if error < min(errors):
                param = idx
        else:
            param = idx
        errors.append(error)

    history.append(errors)

    selected_model = model_fit(train_cv_X,train_cv_y, configs[param])
    selected_model.reset_state()
    predicted = selected_model.predict(test_cv_X)
    if GPU:
        predicted = np.asnumpy(predicted)
    error = np.sqrt(MSE_metric(predicted, test_cv_y))
    best_error.append(error)

    # model eval
    print(f'train-size:{train_X.shape[0]}, val-size:{val_X.shape[0]}, test-size:{test_cv_X.shape[0]}')
    print(f'best_model => error(rmse) : {error}, param:{configs[param]}')
    print()

config : epochs, batch_size, learning_rate
fold : 1/3
 == train [30, 32, 0.01] model ==   error(RMSE):33.79165057554351
 == train [30, 32, 0.001] model ==   error(RMSE):2.0148279949444663
 == train [30, 64, 0.01] model ==   error(RMSE):28.048165052468228
 == train [30, 64, 0.001] model ==   error(RMSE):1.9316286215400762
train-size:80, val-size:102, test-size:102
best_model => error(rmse) : 2.4876631821712945, param:[30, 64, 0.001]

fold : 2/3
 == train [30, 32, 0.01] model ==   error(RMSE):0.6178486529203194
 == train [30, 32, 0.001] model ==   error(RMSE):0.44641865295644456
 == train [30, 64, 0.01] model ==   error(RMSE):29.585626083716495
 == train [30, 64, 0.001] model ==   error(RMSE):0.8786802965057463
train-size:182, val-size:102, test-size:102
best_model => error(rmse) : 0.6358523182490049, param:[30, 32, 0.001]

fold : 3/3
 == train [30, 32, 0.01] model ==   error(RMSE):28.708894105334817
 == train [30, 32, 0.001] model ==   error(RMSE):0.8363687461030129
 == train [30, 64, 0

In [7]:
model_evaluation = sum(best_error)
model_evaluation /= n_splits
print(f'evaluation [Mean RMSE] : {model_evaluation}')

selected_model.reset_state()
predicted = selected_model.predict(test_cv_X)
if GPU:
    predicted = np.asnumpy(predicted)
print(f'MSE : {MSE_metric(predicted, test_cv_y)}')
print(f'RMSE : {np.sqrt(MSE_metric(predicted, test_cv_y))}')

def MAE_metric(x, t):
    return np.mean(numpy.abs(x-t))
print(f'MAE : {MAE_metric(predicted, test_cv_y)}')

print(f'Standard Deviation : {np.std(predicted)}')

evaluation [Mean RMSE] : 1.7940250577777463
MSE : 5.101091796108605
RMSE : 2.2585596729129396
MAE : 2.25635587565282
Standard Deviation : 0.1013544574379921
